# CEUS notebook


## Étape 1 — Chargement du DICOM

Import des bibliothèques et lecture du fichier DICOM avec métadonnées de base.

In [ ]:
# Imports de base
from pathlib import Path
import pydicom
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Définissez le chemin vers votre DICOM
dicom_path = Path('../data')  # Ajustez si besoin

# Si un dossier est fourni, on prend le premier fichier dedans
if dicom_path.is_dir():
    files = sorted([p for p in dicom_path.glob('*') if p.is_file()])
    if not files:
        raise FileNotFoundError(f"Aucun fichier trouvé dans le dossier: {dicom_path}")
    dicom_path = files[0]
print(f"Chemin DICOM sélectionné: {dicom_path}")

In [ ]:
# Lecture du DICOM et impression de quelques métadonnées clés
ds = pydicom.dcmread(str(dicom_path), force=True)
meta = {
    'Rows': getattr(ds, 'Rows', None),
    'Columns': getattr(ds, 'Columns', None),
    'NumberOfFrames': getattr(ds, 'NumberOfFrames', None),
    'PhotometricInterpretation': getattr(ds, 'PhotometricInterpretation', None),
    'CineRate': getattr(ds, 'CineRate', None),
    'FrameTime': getattr(ds, 'FrameTime', None),
    'RecommendedDisplayFrameRate': getattr(ds, 'RecommendedDisplayFrameRate', None),
}
meta

# Extraction du tableau de pixels
arr = ds.pixel_array
print(f"✓ Array extrait: shape={arr.shape}, dtype={arr.dtype}")

In [ ]:
# Helper: conversion YCbCr (YBR) -> RGB (BT.601)
def ycbcr_to_rgb(ycbcr):
    y = ycbcr[:, :, 0].astype(float)
    cb = ycbcr[:, :, 1].astype(float)
    cr = ycbcr[:, :, 2].astype(float)
    r = y + 1.402 * (cr - 128)
    g = y - 0.344136 * (cb - 128) - 0.714136 * (cr - 128)
    b = y + 1.772 * (cb - 128)
    rgb = np.stack([np.clip(r, 0, 255), np.clip(g, 0, 255), np.clip(b, 0, 255)], axis=-1)
    return rgb.astype(np.uint8)

## Étape 1b — Informations scanner

Extraction des métadonnées du scanner (Manufacturer, Model, Institution).


In [ ]:
# Infos scanner standard (DICOM)
scanner_info = {
    "Manufacturer": getattr(ds, "Manufacturer", None),
    "ManufacturerModelName": getattr(ds, "ManufacturerModelName", None),
    "InstitutionName": getattr(ds, "InstitutionName", None),
}
scanner_info

## Étape 2 — Extraction des coordonnées des régions US

Utilisation de la séquence DICOM `SequenceOfUltrasoundRegions` (0018,6011) pour obtenir les bounding boxes.


In [ ]:
# Récupération des régions (US Regions Sequence) et listing des coordonnées
regions = getattr(ds, 'SequenceOfUltrasoundRegions', None)
summary = []
if regions is not None:
    for i, reg in enumerate(regions):
        entry = {
            'index': i,
            'RegionSpatialFormat': getattr(reg, 'RegionSpatialFormat', None),
            'RegionDataType': getattr(reg, 'RegionDataType', None),
            'RegionFlags': getattr(reg, 'RegionFlags', None),
            'MinX0': getattr(reg, 'RegionLocationMinX0', None),
            'MinY0': getattr(reg, 'RegionLocationMinY0', None),
            'MaxX1': getattr(reg, 'RegionLocationMaxX1', None),
            'MaxY1': getattr(reg, 'RegionLocationMaxY1', None),
        }
        summary.append(entry)
summary if regions is not None else 'Aucune SequenceOfUltrasoundRegions trouvée'

## Étape 3 — Identification B-mode / CEUS et extraction des stacks

Classification automatique des régions par analyse de variance inter-canaux (CEUS a un overlay couleur, B-mode est grayscale).


In [ ]:
# Classification et extraction des stacks B-mode / CEUS
import numpy as np
import matplotlib.pyplot as plt

def channel_diff_score(img):
    """Score de couleur: plus élevé si l'image contient de la couleur (pas grayscale)"""
    if img.ndim == 3 and img.shape[-1] == 3:
        r, g, b = img[...,0].astype(float), img[...,1].astype(float), img[...,2].astype(float)
        return np.mean(np.abs(r-g) + np.abs(r-b) + np.abs(g-b))
    return 0.0

def classify_bmode_ceus(crops):
    """Classifier B-mode vs CEUS par analyse de variance inter-canaux"""
    if not crops:
        return None, None
    scores = [channel_diff_score(c['data']) for c in crops]
    ceus_idx = int(np.argmax(scores))  # CEUS a le score couleur le plus élevé
    bmode_idx = None
    if len(crops) > 1:
        bmode_idx = 1 - ceus_idx if len(crops) == 2 else int(np.argmin(scores))
    return bmode_idx, ceus_idx

def crop_regions_from_frame(frame, regions):
    """Extraire les régions depuis un frame en utilisant les métadonnées US Regions"""
    h, w = frame.shape[0], frame.shape[1]
    crops = []
    if regions is None:
        return crops
    
    for i, reg in enumerate(regions):
        x0 = int(getattr(reg, 'RegionLocationMinX0', 0) or 0)
        y0 = int(getattr(reg, 'RegionLocationMinY0', 0) or 0)
        x1 = int(getattr(reg, 'RegionLocationMaxX1', w-1) or (w-1))
        y1 = int(getattr(reg, 'RegionLocationMaxY1', h-1) or (h-1))
        x0, y0 = max(0, x0), max(0, y0)
        x1, y1 = min(w-1, x1), min(h-1, y1)
        
        crop = frame[y0:y1+1, x0:x1+1, ...] if frame.ndim == 3 else frame[y0:y1+1, x0:x1+1]
        
        crops.append({
            'index': i,
            'bbox': (x0, y0, x1, y1),
            'data': crop,
        })
    return crops

def crop_stack(arr, bbox, photo):
    """Crop un stack complet avec conversion YBR→RGB si nécessaire"""
    x0, y0, x1, y1 = bbox
    
    if arr.ndim == 4 and arr.shape[-1] == 3:  # (T, H, W, C)
        sub = arr[:, y0:y1+1, x0:x1+1, :]
        if 'YBR' in (photo or ''):
            out = np.empty_like(sub)
            for i in range(sub.shape[0]):
                out[i] = ycbcr_to_rgb(sub[i])
            return out
        return sub
    elif arr.ndim == 3:
        if arr.shape[0] < arr.shape[1]:  # (T, H, W) grayscale
            return arr[:, y0:y1+1, x0:x1+1]
        else:  # (H, W, C) image unique
            img = arr[y0:y1+1, x0:x1+1, :]
            if img.ndim == 3 and img.shape[-1] == 3 and 'YBR' in (photo or ''):
                img = ycbcr_to_rgb(img)
            return img[np.newaxis, ...]
    else:  # (H, W)
        return arr[y0:y1+1, x0:x1+1][np.newaxis, ...]

# ─────────────────────────────────────────────────────────────
# Extraction et classification
# ─────────────────────────────────────────────────────────────

# Préparer le premier frame pour analyse
frame0 = arr[0] if arr.ndim == 4 or (arr.ndim == 3 and arr.shape[0] < arr.shape[1]) else arr
photo = getattr(ds, 'PhotometricInterpretation', '') or ''
frame0_rgb = ycbcr_to_rgb(frame0) if (frame0.ndim == 3 and frame0.shape[-1] == 3 and 'YBR' in photo) else frame0

# Extraire les crops des régions
crops = crop_regions_from_frame(frame0_rgb, regions)

if not crops:
    print("⚠️ Aucune région détectée dans les métadonnées DICOM")
    print("   Le fichier ne contient pas de SequenceOfUltrasoundRegions")
    bmode_idx, ceus_idx = None, None
    bmode_stack, ceus_stack = None, None
else:
    # Classifier les régions
    bmode_idx, ceus_idx = classify_bmode_ceus(crops)
    
    print(f'✓ Classification: B-mode=région {bmode_idx}, CEUS=région {ceus_idx}')
    print(f'  Score couleur région 0: {channel_diff_score(crops[0]["data"]):.1f}')
    if len(crops) > 1:
        print(f'  Score couleur région 1: {channel_diff_score(crops[1]["data"]):.1f}')
    
    # Extraire les stacks complets
    bmode_stack = crop_stack(arr, crops[bmode_idx]['bbox'], photo) if bmode_idx is not None else None
    ceus_stack = crop_stack(arr, crops[ceus_idx]['bbox'], photo) if ceus_idx is not None else None
    
    print(f'\n✓ Stacks extraits:')
    print(f'  B-mode: {bmode_stack.shape if bmode_stack is not None else "None"}')
    print(f'  CEUS:   {ceus_stack.shape if ceus_stack is not None else "None"}')
    
    # Affichage côte-à-côte
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    if bmode_stack is not None and bmode_stack.size > 0:
        img_b = bmode_stack[0]
        axes[0].imshow(img_b if (img_b.ndim == 3 and img_b.shape[-1] == 3) else img_b, 
                      cmap=None if (img_b.ndim == 3 and img_b.shape[-1] == 3) else 'gray')
        axes[0].set_title(f'B-mode (Région {bmode_idx})', fontsize=14, fontweight='bold')
        axes[0].axis('off')
    
    if ceus_stack is not None and ceus_stack.size > 0:
        img_c = ceus_stack[0]
        axes[1].imshow(img_c if (img_c.ndim == 3 and img_c.shape[-1] == 3) else img_c,
                      cmap=None if (img_c.ndim == 3 and img_c.shape[-1] == 3) else 'gray')
        axes[1].set_title(f'CEUS (Région {ceus_idx})', fontsize=14, fontweight='bold')
        axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()


## Étape 4 — Détection du flash CEUS

Détection robuste du flash par analyse du gradient négatif (chute d'intensité après destruction des microbulles).
Les premières frames sont exclues pour éviter les artefacts d'acquisition.

In [ ]:
# Détection robuste du flash CEUS
import numpy as np
import matplotlib.pyplot as plt

def detect_flash_ceus(ceus_stack, exclude_first_n=5, method='gradient'):
    """
    Détection du flash par analyse du gradient négatif
    
    Args:
        ceus_stack: array (T, H, W, C) ou (T, H, W)
        exclude_first_n: nombre de frames initiales à exclure (artefacts)
        method: 'gradient' (chute après flash) ou 'peak' (pic d'intensité)
    
    Returns:
        flash_idx: index de la frame du flash
        intensities: courbe d'intensité moyenne par frame
    """
    # Calculer l'intensité moyenne par frame
    if ceus_stack.ndim == 4:  # (T, H, W, C)
        intensities = ceus_stack.mean(axis=(1, 2, 3))
    else:  # (T, H, W)
        intensities = ceus_stack.mean(axis=(1, 2))
    
    T = len(intensities)
    start_frame = min(exclude_first_n, T // 10)
    
    if method == 'gradient':
        # Chercher la plus grande chute (gradient négatif maximal)
        # Le flash est la frame AVANT la chute
        gradients = np.diff(intensities)
        flash_idx = start_frame + np.argmin(gradients[start_frame:])
        detection_method = "gradient négatif maximal"
        
    elif method == 'peak':
        # Chercher le pic d'intensité (après exclusion des premières frames)
        flash_idx = start_frame + np.argmax(intensities[start_frame:])
        detection_method = "pic d'intensité"
    
    return flash_idx, intensities, detection_method


# ─────────────────────────────────────────────────────────────
# Détection du flash
# ─────────────────────────────────────────────────────────────

if ceus_stack is not None:
    # Détecter le flash avec les deux méthodes
    flash_idx_grad, intensities, method_grad = detect_flash_ceus(ceus_stack, exclude_first_n=5, method='gradient')
    flash_idx_peak, _, method_peak = detect_flash_ceus(ceus_stack, exclude_first_n=5, method='peak')
    
    # Utiliser la méthode gradient par défaut (plus robuste)
    flash_idx = flash_idx_grad
    
    print(f"✓ Flash détecté:")
    print(f"  Méthode gradient: frame {flash_idx_grad} (0-indexed)")
    print(f"  Méthode peak:     frame {flash_idx_peak} (0-indexed)")
    print(f"\n→ Flash retenu: frame {flash_idx} ({method_grad})")
    
    # ─────────────────────────────────────────────────────────────
    # Visualisation
    # ─────────────────────────────────────────────────────────────
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 8))
    
    # 1. Courbe d'intensité complète
    axes[0, 0].plot(intensities, 'b-', linewidth=2, label='Intensité moyenne')
    axes[0, 0].axvline(flash_idx_grad, color='red', linestyle='--', linewidth=2, label=f'Flash (gradient) = {flash_idx_grad}')
    axes[0, 0].axvline(flash_idx_peak, color='orange', linestyle=':', linewidth=2, label=f'Peak = {flash_idx_peak}')
    axes[0, 0].axvspan(0, 5, alpha=0.2, color='gray', label='Zone exclue')
    axes[0, 0].set_xlabel('Frame', fontsize=12)
    axes[0, 0].set_ylabel('Intensité moyenne', fontsize=12)
    axes[0, 0].set_title('Courbe d\'intensité CEUS', fontsize=14, fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Zoom autour du flash
    window = 10
    start = max(0, flash_idx - window)
    end = min(len(intensities), flash_idx + window)
    axes[0, 1].plot(range(start, end), intensities[start:end], 'b-o', linewidth=2, markersize=6)
    axes[0, 1].axvline(flash_idx, color='red', linestyle='--', linewidth=2, label=f'Flash = {flash_idx}')
    axes[0, 1].set_xlabel('Frame', fontsize=12)
    axes[0, 1].set_ylabel('Intensité moyenne', fontsize=12)
    axes[0, 1].set_title(f'Zoom autour du flash (±{window} frames)', fontsize=14, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Frame avant le flash
    idx_before = max(0, flash_idx - 1)
    img_before = ceus_stack[idx_before]
    if img_before.ndim == 3 and img_before.shape[-1] == 3:
        axes[1, 0].imshow(img_before)
    else:
        axes[1, 0].imshow(img_before, cmap='gray')
    axes[1, 0].set_title(f'Frame {idx_before} (avant flash)', fontsize=12, fontweight='bold')
    axes[1, 0].axis('off')
    
    # 4. Frame après le flash
    idx_after = min(len(ceus_stack) - 1, flash_idx + 1)
    img_after = ceus_stack[idx_after]
    if img_after.ndim == 3 and img_after.shape[-1] == 3:
        axes[1, 1].imshow(img_after)
    else:
        axes[1, 1].imshow(img_after, cmap='gray')
    axes[1, 1].set_title(f'Frame {idx_after} (après flash)', fontsize=12, fontweight='bold')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️ Aucun stack CEUS disponible pour la détection du flash")


## Étape 5 — Crop temporel : Flash + 15 secondes

Extraction de la fenêtre d'analyse post-flash (phase de reperfusion).

In [ ]:
# Crop temporel : flash + 15 secondes
import numpy as np

if ceus_stack is not None and flash_idx is not None:
    # Calcul du nombre de frames pour 15 secondes
    # Essayer d'abord FrameTime (en ms), sinon CineRate, sinon RecommendedDisplayFrameRate
    fps = None
    
    if meta.get('FrameTime') is not None:
        frame_time_ms = meta['FrameTime']
        fps = 1000.0 / frame_time_ms
        source = "FrameTime"
    elif meta.get('CineRate') is not None:
        fps = meta['CineRate']
        source = "CineRate"
    elif meta.get('RecommendedDisplayFrameRate') is not None:
        fps = meta['RecommendedDisplayFrameRate']
        source = "RecommendedDisplayFrameRate"
    else:
        # Fallback : estimer à partir du nombre total de frames (hypothèse : acquisition de 60s)
        fps = len(ceus_stack) / 60.0
        source = "estimation (60s total)"
    
    # Calculer le nombre de frames pour 15 secondes
    duration_s = 15
    frames_15s = int(duration_s * fps)
    
    # Index de début et fin
    start_idx = flash_idx
    end_idx = min(flash_idx + frames_15s, len(ceus_stack))
    
    # Crop temporel
    ceus_cropped = ceus_stack[start_idx:end_idx]
    
    # Résumé
    print(f"✓ Crop temporel effectué:")
    print(f"  FPS: {fps:.2f} ({source})")
    print(f"  Durée demandée: {duration_s}s")
    print(f"  Frames pour {duration_s}s: {frames_15s}")
    print(f"  Flash à la frame: {flash_idx}")
    print(f"  Crop: frames [{start_idx}:{end_idx}] ({end_idx - start_idx} frames)")
    print(f"  Durée réelle: {(end_idx - start_idx) / fps:.2f}s")
    print(f"\n  Shape avant crop: {ceus_stack.shape}")
    print(f"  Shape après crop: {ceus_cropped.shape}")
    
else:
    print("⚠️ Impossible de faire le crop temporel (stack CEUS ou flash_idx manquant)")
    ceus_cropped = None


## Étape 6 — Visualisation des frames croppées

Affichage d'un échantillon des frames de la phase de reperfusion.

In [ ]:
# Visualisation d'un échantillon des frames croppées
import matplotlib.pyplot as plt
import numpy as np

if ceus_cropped is not None and len(ceus_cropped) > 0:
    # Sélectionner 6 frames représentatives (début, milieu, fin)
    n_frames = len(ceus_cropped)
    if n_frames >= 6:
        # Échantillonnage uniforme sur toute la séquence
        indices = np.linspace(0, n_frames - 1, 6, dtype=int)
    else:
        # Toutes les frames disponibles
        indices = np.arange(n_frames)
    
    n_display = len(indices)
    
    # Affichage en grille 2×3
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, frame_idx in enumerate(indices):
        img = ceus_cropped[frame_idx]
        
        # Afficher l'image
        if img.ndim == 3 and img.shape[-1] == 3:
            axes[idx].imshow(img)
        else:
            axes[idx].imshow(img, cmap='gray')
        
        # Calculer le temps relatif depuis le flash
        time_s = frame_idx / fps if fps else 0
        axes[idx].set_title(f'Frame {frame_idx} (t = {time_s:.2f}s)', fontsize=12, fontweight='bold')
        axes[idx].axis('off')
    
    # Masquer les subplots inutilisés
    for idx in range(n_display, 6):
        axes[idx].axis('off')
    
    plt.suptitle('Échantillon de frames CEUS (phase de reperfusion)', fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()
    
    print(f"✓ Affichage de {n_display} frames représentatives sur {n_frames} frames totales")
    
else:
    print("⚠️ Aucune frame croppée disponible pour l'affichage")
